In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd            
import re
from tqdm import tqdm
from datetime import datetime

In [2]:
url = 'https://www.imdb.com/search/title/?title_type=tv_series&release_date=2019-10-10,&genres=drama&count=250&ref_=adv_prv'
url_text = requests.get(url).text
url_soup = BeautifulSoup(url_text, 'html.parser')

In [4]:
template = 'https://www.imdb.com%s'

# get the title links for the tv series
title_links = [template % a.attrs.get('href') for a in url_soup.select('a')]

imdb_show_list = []

for i in tqdm(range(0, len(title_links)), desc="Shows processed", ncols=100):
    page_url = title_links[i]
    page_text = requests.get(page_url).text
    page_soup = BeautifulSoup(page_text, 'html.parser')
    
    show_name = page_soup.find("div",{"class":"col-title"}).get_text()
    year = page_soup.find("span",{"span":"lister-item-year.text-muted.unbold"}).get_text()
    rating = page_soup.find("div",{"class":"inline-block ratings-imdb-rating"}).text
    vote_count = page_soup.find("span",{"nv":"data-value"}).text
    release_date = ((page_soup.find("a",{"class":"ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"}).get_text(strip=True).split('|')[0]).split('(')[1]).split(')')[0]
    service = (page_soup.find("ul",{"class":"ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base"}).get_text(strip=True).split("|")) 
    
    imdb_dict = {'show_name': show_name, 
                 'year': year, 
                 'rating': rating,
                 'vote_count': vote_count,
                 'release_date': release_date,
                 'service': service}
    
    imdb_show_list.append(show_dict)

Shows processed:   0%|                                                     | 0/4085 [00:11<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.imdb_db
collection = db.recent

In [ ]:
# URL of page to be scraped
url = 'https://www.imdb.com/search/title/?title_type=tv_series&release_date=2019-10-10,&genres=drama&count=250'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='lister-item-mode-advanced')

# loop over results to get article data
for result in results:
    # scrape the title  
    title = result.find('h3', 'a[href]').text
    
    # scrape the year released
    year = result.find('span', class_='lister-item-year text-muted unbold').text
    
    # scrape the imdb score
    imdb_score = result.find('div', class_='data-value').text 
    
    # get the number of votes
    votes = results.find('span', class_='data-value').text
    
    # print article data
    print('-----------------')
    print(title)
    print(year)
    print(imdb_score)
    print(votes)

    # Dictionary to be inserted into MongoDB
    post = {
        'title': title,
        'year': year,
        'imdb score': imdb_score,
        'votes': votes
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)

In [ ]:
# Display the MongoDB records created above
recent = db.recent.find()
for recent in recent:
    print(recent)